# Disaster tweets DL model

In [4]:
import numpy as np
import pandas as pd
from sklearn import feature_extraction, ensemble, model_selection, pipeline, compose, preprocessing, metrics
from sklearn.experimental import enable_halving_search_cv
import tensorflow as tf
SCRIPT_NAME='DL-02'

In [3]:
df_train = pd.read_csv('./train_enriched.csv', index_col='id')
df_train.fillna({'keyword': '', 'location': '', 'country': '', 'state': '', 'city': ''}, inplace=True)
df_train.head()

__main__


,keyword,positive_factor,location,country,state,city,missing_location,text,clean_text,text_length,upper_text_factor,tags_count,punct_factor,ann_count,urls_count,tokens_count,stop_words_factor,clean_tokens_factor,target
id,,,,,,,,,,,,,,,,,,,
0,,0.5,,,,,1,Our Deeds are the Reason of this #earthquake M...,deed reason earthquake may allah forgive u,57,0.175439,1,0.017544,0,0,13,0.384615,0.615385,1
1,,0.5,,,,,1,Forest fire near La Ronge Sask. Canada,forest fire near la ronge sask canada,32,0.156250,0,0.031250,0,0,7,0.000000,1.000000,1
2,,0.5,,,,,1,All residents asked to 'shelter in place' are ...,resident asked shelter place notified officer ...,112,0.017857,0,0.026786,0,0,22,0.409091,0.590909,1
3,,0.5,,,,,1,"13,000 people receive #wildfires evacuation or...",people receive wildfire evacuation order calif...,57,0.017544,1,0.035088,0,0,9,0.111111,0.888889,1
4,,0.5,,,,,1,Just got sent this photo from Ruby #Alaska as ...,got sent photo ruby alaska smoke wildfire pour...,72,0.041667,2,0.027778,0,0,17,0.352941,0.647059,1


In [52]:
categorical_features = [
    'country',
    'state',
]
numerical_features = [
    'text_length', 
    'urls_count',
    'stop_words_factor',
    'clean_tokens_factor',
    'positive_factor'
]

vc_text = feature_extraction.text.TfidfVectorizer(max_features=2000)

transformer = compose.ColumnTransformer(transformers=[
    ('text_vector', vc_text, 'clean_text'),
    ('one_hot', preprocessing.OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features),
    ('numerical', preprocessing.StandardScaler(), numerical_features)
], remainder='drop')

X_train = transformer.fit_transform(df_train).todense()
print('X_train shape', X_train.shape)

Y_train = df_train['target']

X_train.shape, Y_train.shape


X_train shape (7613, 2172)


((7613, 2172), (7613,))

In [83]:
INPUT_SIZE = X_train.shape[1]
OUTPUT_SIZE = 1
HIDDEN_LAYER_SIZE = 1024
BATCH_SIZE= 64
MAX_EPOCHS = 200

def baseline_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2*HIDDEN_LAYER_SIZE, input_shape=(INPUT_SIZE,), activation='relu'),
        tf.keras.layers.Dense(2*HIDDEN_LAYER_SIZE, activation='relu'),
        tf.keras.layers.Dense(HIDDEN_LAYER_SIZE, activation='relu'),
        tf.keras.layers.Dense(OUTPUT_SIZE, activation='linear')
    ])
    model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics='accuracy')
    return model

In [84]:
model = baseline_model()
early_stopping = tf.keras.callbacks.EarlyStopping(patience=4, restore_best_weights=True)
history = model.fit(X_train, Y_train, batch_size=BATCH_SIZE, epochs=MAX_EPOCHS, validation_split=0.2, 
          callbacks=[early_stopping],
          verbose=2)

Y_predict = tf.cast(tf.greater(tf.nn.sigmoid(model(X_train)), .5), tf.int32)

df_wrong_predictions = pd.DataFrame({'target': tf.squeeze(Y_train), 'predict': tf.squeeze(Y_predict), 'keyword': df_train['keyword'], 'location': df_train['location'], 'text': df_train['text']}).query('target != predict')

f1_score = metrics.f1_score(Y_train, Y_predict)
accuracy = metrics.accuracy_score(Y_train, Y_predict)
precision = metrics.precision_score(Y_train, Y_predict)

print(f'F1={f1_score}, accuracy={accuracy}, precision={precision}')


Epoch 1/200
96/96 - 7s - loss: 0.4968 - accuracy: 0.7573 - val_loss: 0.4800 - val_accuracy: 0.7452 - 7s/epoch - 73ms/step
Epoch 2/200
96/96 - 9s - loss: 0.3424 - accuracy: 0.8440 - val_loss: 0.4415 - val_accuracy: 0.8076 - 9s/epoch - 95ms/step
Epoch 3/200
96/96 - 9s - loss: 0.2000 - accuracy: 0.9174 - val_loss: 0.6120 - val_accuracy: 0.7794 - 9s/epoch - 90ms/step
Epoch 4/200
96/96 - 8s - loss: 0.1024 - accuracy: 0.9599 - val_loss: 0.7085 - val_accuracy: 0.7807 - 8s/epoch - 85ms/step
Epoch 5/200
96/96 - 8s - loss: 0.0775 - accuracy: 0.9678 - val_loss: 1.0827 - val_accuracy: 0.7905 - 8s/epoch - 82ms/step
Epoch 6/200
96/96 - 8s - loss: 0.0559 - accuracy: 0.9752 - val_loss: 1.0593 - val_accuracy: 0.7853 - 8s/epoch - 83ms/step
F1=0.8888186986734049, accuracy=0.9075265992381453, precision=0.9193074158771644


In [85]:
with pd.option_context('display.max_colwidth', 200):
    print(df_wrong_predictions.query('target==0').sample(n=10))

      target  predict            keyword                  location  \
id                                                                   
2066       0        1               dead                             
7386       0        1          windstorm                   Houston   
6868       0        1             trauma                             
7317       0        1       wild%20fires      West Vancouver, B.C.   
2546       0        1            destroy             Johannesburg    
1190       0        1  bridge%20collapse                 Leicester   
4344       0        1             hijack                   Nigeria   
2389       0        1             derail           London, England   
2503       0        1           desolate                             
7092       0        1           upheaval  Perth, Western Australia   

                                                                                                                                                text  
id      

#### Generate output

In [86]:
model.save('C-DL-1')

INFO:tensorflow:Assets written to: C-DL-1/assets


INFO:tensorflow:Assets written to: C-DL-1/assets


In [87]:
X_test = transformer.transform(pd.read_csv('./test_enriched.csv', index_col='id')).todense()
print('X_test shape', X_test.shape)

Y_test_predict = tf.cast(tf.greater(tf.nn.sigmoid(model(X_test)), .5), tf.int32)

df_example = pd.read_csv('./sample_submission.csv')
df_example['target'] = Y_test_predict

df_example.to_csv('./C-DL-1-submission.csv', index=False)

X_test shape (3263, 2172)
